# Week 4 - Analysis of TSP using Hill Climbing Algorithms
## Name: Rakshit Ramachandra Ayachit
## Registration No.: 210968045
## Batch: B1
## Section: DSE-A


In [12]:
# Function to Load TSP Data from File 
def load_data(filename):
    with open(filename, "r") as f:
        V = []
        data = f.readlines()
        for i, line in enumerate(data):
            if line.startswith("NODE_COORD_SECTION"):
                data = data[i+1:len(data)-1]
                break
        for i, line in enumerate(data):
            node = line.split()[1:]
            node = np.array([float(i) for i in node])
            V.append(node)
        cities = {i : V[i] for i in range(len(V))}
    return cities

In [13]:
# Function to Calculate Euclidean Distance between Two Points
def euclid_distance(x,y):
    # x and y are numpy arrays
    return np.sqrt(np.sum((x-y)**2))

# Function to Calculate Paths between Cities
def calculate_paths(cities:dict):
    names = [i for i in range(len(cities))]
    paths = np.zeros((len(names), len(names)))
    for i in range(len(names)):
        for j in range(len(names)):
            if i != j:
                paths[i,j] = euclid_distance(cities[names[i]], cities[names[j]])
    return paths

In [14]:
# Function to Initialize a Tour
def initialize_tour(paths):

    num_cities = paths.shape[0]
    visited = [False] * num_cities
    tour = [np.random.randint(0, num_cities)]
    visited[tour[0]] = True

    while len(tour) < num_cities:
        current_city = tour[-1]
        nearest_city = None
        nearest_distance = float('inf')

        # Find the nearest unvisited city to the current city
        for city in range(num_cities):
            if not visited[city] and paths[current_city, city] < nearest_distance:
                nearest_city = city
                nearest_distance = paths[current_city, city]

        tour.append(nearest_city)
        visited[nearest_city] = True

    return tour

In [15]:
# Function to Generate Neighboring Tours
def generate_neighbors(x, n=10):
    neighbors = []
    while len(neighbors) < n:
        i = np.random.randint(0, len(x)-2)
        j = np.random.randint(1, len(x)-1)
        if i > j :
            i, j = j, i
        neighbor = x.copy()
        neighbor[i:j] = neighbor[i:j][::-1]
        if tuple(neighbor) not in map(tuple, neighbors):
            neighbors.append(neighbor)
    return neighbors

In [16]:
import numpy as np
# Function to Compute Fitness of a Tour
def fitness(x, paths):
    # x is a list of cities
    fitness = 0
    for i in range(len(x)-1):
        fitness += paths[x[i], x[i+1]]
    fitness += paths[x[-1], x[0]]
    return fitness

# Function to Find the Best Neighbor
def best_neighbor(x:list, paths:np.array, generate_neighbors:callable = generate_neighbors, fitness: callable = fitness):
    neighbors = generate_neighbors(x)
    best_neighbor = neighbors[0]
    for neighbor in range(1, len(neighbors)):
        if fitness(neighbors[neighbor], paths) < fitness(best_neighbor, paths):
            best_neighbor = neighbors[neighbor]
    return best_neighbor

# Function to Generate a Random Neighbor
def random_neighbor(x:list, paths:np.array, generate_neighbors:callable = generate_neighbors):
    neighbors = generate_neighbors(x)
    return neighbors[np.random.randint(0, len(neighbors))]

In [17]:
# Function for Hill Climbing Algorithm
def hill_climbing(f: callable, x_init: list, n_iters: int, paths: np.array, variant: str, steepest: bool = False):
    x = x_init
    x_best = x
    if variant == "simple":
        neighbor_function = best_neighbor
    elif variant == "stochastic":
        neighbor_function = random_neighbor

    tour_lengths = []
    for iter in tqdm(range(n_iters)):
        y = neighbor_function(x, paths)
        if f(x, paths) > f(y, paths):
            x = y
            if f(x, paths) < f(x_best, paths):
                x_best = x
        elif steepest:
            x = x_best
        
        tour_lengths.append(f(x, paths))

    return x_best, tour_lengths


In [18]:
# Function to Evaluate Hill Climbing Algorithms
def evaluate_hill_climbing_algorithm(filename):
    cities = load_data(filename)
    paths = calculate_paths(cities)

    initial_tour = initialize_tour(paths)

    # Evaluate using Simple Hill Climbing
    final_tour_simple, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="simple", steepest=False)
    final_length_simple = fitness(final_tour_simple, paths)

    # Evaluate using Stochastic Hill Climbing
    final_tour_stochastic, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="stochastic", steepest=False)
    final_length_stochastic = fitness(final_tour_stochastic, paths)

    # Evaluate using Steepest-Ascent Hill Climbing
    final_tour_steepest, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="simple", steepest=True)
    final_length_steepest = fitness(final_tour_steepest, paths)

    initial_length = fitness(initial_tour, paths)
    
    return initial_length, final_length_simple, final_length_stochastic, final_length_steepest, cities



In [19]:
# Function to Calculate Paths between Cities
def calculate_paths(cities):
    num_cities = len(cities)
    paths = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                paths[i, j] = euclid_distance(cities[i], cities[j])
    return paths

In [20]:
from tqdm import tqdm
def main():
    # Define datasets
    datasets = ["rd100.tsp", "eil101.tsp", "a280.tsp", "d198.tsp", "ch150.tsp"]

    # Results dictionary to store evaluation results
    results = {}

    # Evaluate each dataset using hill climbing algorithms
    for dataset in datasets:
        initial_length, final_length_simple, final_length_stochastic, final_length_steepest, cities = evaluate_hill_climbing_algorithm(dataset)
        results[dataset] = {
            "Initial Length": initial_length,
            "Final Length (Simple)": final_length_simple,
            "Final Length (Stochastic)": final_length_stochastic,
            "Final Length (Steepest)": final_length_steepest
        }
    
    print("Results:")
    
    print("Dataset\t\tInitial Length\tFinal Length (Simple)\tFinal Length (Stochastic)\tFinal Length (Steepest)")
    for dataset, values in results.items():
        print("{:<10}\t{:<15}\t{:<22}\t{:<25}\t{:<20}".format(dataset, 
                                                        values['Initial Length'], 
                                                        values['Final Length (Simple)'],
                                                        values['Final Length (Stochastic)'],
                                                        values['Final Length (Steepest)']))


    distance_matrix = calculate_paths(cities)  # Make sure 'cities' is defined somewhere
    print("\nDistance Matrix:")
    print(distance_matrix)

    # Tabulate results
    print("Final Results:")
    print("Dataset\t\tBest Local Search Method")
    for dataset, lengths in results.items():
        best_method = min(lengths, key=lengths.get)
        print(f"{dataset}\t\t{best_method}")


if __name__ == "__main__":
    main()


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1277.14it/s]


Results:
Dataset		Initial Length	Final Length (Simple)	Final Length (Stochastic)	Final Length (Steepest)
rd100.tsp 	10314.533729243238	9004.508685888091     	9968.044976731557        	8665.519897358974   
eil101.tsp	772.4912382688786	702.6706794163167     	751.3756969408782        	699.6499243703689   
a280.tsp  	3201.914719805688	3073.3187187015264    	3198.0203293682334       	3121.483588330063   
d198.tsp  	18917.645182586115	18314.31114827744     	18594.295080266722       	18265.52483568615   
ch150.tsp 	7504.355971522763	7242.198291317867     	7386.54805772816         	7038.842999073499   

Distance Matrix:
[[  0.         576.64638554 188.06188449 ... 769.70425494 270.45040003
  378.75411581]
 [576.64638554   0.         591.14787168 ... 368.26459998 606.70343631
  615.23173043]
 [188.06188449 591.14787168   0.         ... 688.45153118  82.7803772
  194.30913097]
 ...
 [769.70425494 368.26459998 688.45153118 ...   0.         657.19430602
  600.16082277]
 [270.45040003 606.70343631 

In Simple Hill Climbing, the algorithm iterates by making small modifications to the current solution and evaluates whether the modified solution is better or worse than the current one.
If the modified solution is better, it replaces the current solution with the modified one.
Simple Hill Climbing may get stuck in local optima, as it only considers the improvement of the current solution.
Stochastic Hill Climbing:

Stochastic Hill Climbing is similar to Simple Hill Climbing, but it allows for some degree of randomness in decision-making.
Instead of always accepting a better solution, Stochastic Hill Climbing probabilistically decides whether to accept a new solution, even if it is worse than the current one.
This randomness helps to avoid getting trapped in local optima and allows exploration of the solution space.
Steepest-Ascent Hill Climbing:

In Steepest-Ascent Hill Climbing, the algorithm evaluates all neighboring solutions from the current one and selects the one that offers the greatest improvement.
Unlike Simple Hill Climbing, which accepts the first solution that offers improvement, Steepest-Ascent Hill Climbing ensures that it always moves to the best neighboring solution.
This approach can be more computationally intensive, as it requires evaluating all neighbors, but it tends to lead to better solutions compared to Simple Hill Climbing.

In [1]:
def evaluate_hill_climbing_algorithm(filename, fitness=None, initial_tour=None):
    cities = load_data(filename)
    paths = calculate_paths(cities)

    if fitness is None:
        fitness = default_fitness_function
    
    if initial_tour is None:
        initial_tour = initialize_tour(paths)

    # Evaluate using Simple Hill Climbing
    final_tour_simple, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="simple", steepest=False)
    final_length_simple = fitness(final_tour_simple, paths)

    # Evaluate using Stochastic Hill Climbing
    final_tour_stochastic, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="stochastic", steepest=False)
    final_length_stochastic = fitness(final_tour_stochastic, paths)

    # Evaluate using Steepest-Ascent Hill Climbing
    final_tour_steepest, _ = hill_climbing(fitness, initial_tour, n_iters=1000, paths=paths, variant="simple", steepest=True)
    final_length_steepest = fitness(final_tour_steepest, paths)

    initial_length = fitness(initial_tour, paths)
    
    return initial_length, final_length_simple, final_length_stochastic, final_length_steepest, cities


In [2]:
from tqdm import tqdm
def main():
    # Define datasets
    datasets = ["rd100.tsp", "eil101.tsp", "a280.tsp", "d198.tsp", "ch150.tsp"]

    # Results dictionary to store evaluation results
    results = {}

    # Evaluate each dataset using hill climbing algorithms
    for dataset in datasets:
        initial_length, final_length_simple, final_length_stochastic, final_length_steepest, cities = evaluate_hill_climbing_algorithm(dataset)
        results[dataset] = {
            "Initial Length": initial_length,
            "Final Length (Simple)": final_length_simple,
            "Final Length (Stochastic)": final_length_stochastic,
            "Final Length (Steepest)": final_length_steepest
        }
    
    print("Results:")
    
    print("Dataset\t\tInitial Length\tFinal Length (Simple)\tFinal Length (Stochastic)\tFinal Length (Steepest)")
    for dataset, values in results.items():
        print("{:<10}\t{:<15}\t{:<22}\t{:<25}\t{:<20}".format(dataset, 
                                                        values['Initial Length'], 
                                                        values['Final Length (Simple)'],
                                                        values['Final Length (Stochastic)'],
                                                        values['Final Length (Steepest)']))


    distance_matrix = calculate_paths(cities)  # Make sure 'cities' is defined somewhere
    print("\nDistance Matrix:")
    print(distance_matrix)

    # Tabulate results
    print("Final Results:")
    print("Dataset\t\tBest Local Search Method")
    for dataset, lengths in results.items():
        best_method = min(lengths, key=lengths.get)
        print(f"{dataset}\t\t{best_method}")


if __name__ == "__main__":
    main()


NameError: name 'load_data' is not defined